In [73]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import re

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\Martin\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [98]:
from unittest import result


def scrape(index):
    driver.get(f'https://www.diffordsguide.com/cocktails/recipe/{index}/')
    driver.add_cookie({'name':'diffordsguide_com_subscribe','value':'1'})
    driver.find_element(by=By.XPATH, value="/html/body/div[3]/div[2]/section/div/div/div/div[1]/div/div").screenshot(f'images/difford/{index}.png')
    image_url = driver.find_element(by=By.XPATH, value='//img[@alt="Product image"]').get_property('src')
    drink_name = driver.find_element(by=By.XPATH, value='/html/body/div[3]/div[2]/div[2]/div/div/div[1]/h1').text
    serving_suggestion = driver.find_element(by=By.CSS_SELECTOR,value='.pad-bottom > .grid-margin-y > .cell:nth-child(1)').text

    text_boxes = driver.find_elements(by=By.CSS_SELECTOR,value='.pad-bottom .cell p')

    garnish = text_boxes[0].text
    instructions = text_boxes[1].text

    ingredients_list = driver.find_element(by=By.CLASS_NAME, value='ingredients-table').text.split('\n')[:-4]

    url = driver.current_url


    try:
        review_text = driver.find_element(by=By.CSS_SELECTOR,value='.always-first-small').text
        review_count = re.search('[0-9]+',review_text).group(0)
    except:
        review_count = 0


    result= [
        drink_name,
        garnish, 
        serving_suggestion,
        ingredients_list,
        instructions, 
        review_count,
        url, 
        'diffords', 
        image_url
    ]

    return result

In [99]:
import pickle
index_list = pickle.load(open('difford_1_5000.p','rb'))
rows = [[]] *int(index_list[-1])

for i in index_list:
    try:
        new_row = scrape(i)
        rows[i-1] = new_row
    except Exception as e:
        print(f'Index {i} threw an exception, skipping.')
        print(e)
df = pd.DataFrame(rows, columns=['name','garnish','serving_suggestion', 'ingredients','steps','review_count','url', 'website', 'image_url']).dropna()
df.to_csv('./difford_1_5000.csv', index=False)

Index 25 threw an exception, skipping.
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/section/div/div/div/div[1]/div/div"}
  (Session info: chrome=102.0.5005.115)
Stacktrace:
Backtrace:
	Ordinal0 [0x00FBD953+2414931]
	Ordinal0 [0x00F4F5E1+1963489]
	Ordinal0 [0x00E3C6B8+837304]
	Ordinal0 [0x00E69500+1021184]
	Ordinal0 [0x00E6979B+1021851]
	Ordinal0 [0x00E96502+1205506]
	Ordinal0 [0x00E844E4+1131748]
	Ordinal0 [0x00E94812+1198098]
	Ordinal0 [0x00E842B6+1131190]
	Ordinal0 [0x00E5E860+976992]
	Ordinal0 [0x00E5F756+980822]
	GetHandleVerifier [0x0122CC62+2510274]
	GetHandleVerifier [0x0121F760+2455744]
	GetHandleVerifier [0x0104EABA+551962]
	GetHandleVerifier [0x0104D916+547446]
	Ordinal0 [0x00F55F3B+1990459]
	Ordinal0 [0x00F5A898+2009240]
	Ordinal0 [0x00F5A985+2009477]
	Ordinal0 [0x00F63AD1+2046673]
	BaseThreadInitThunk [0x76756739+25]
	RtlGetFullPathName_UEx [0x77BC8AFF+1215]
	RtlGetFullPathName_UEx [0x77BC8ACD+1165]

Index 413 th

In [101]:
df

,name,garnish,serving_suggestion,ingredients,steps,review_count,url,website,image_url
0,Abacaxi Ricaço,Cut a straw sized hole in the top of the pinea...,Serve in a Pineapple shell (frozen) glass,"[1 whole Pineapple (fresh), 3 shot Havana Club...",Cut the top off a small pineapple and carefull...,4,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...
2,Abbey,Orange zest twist,Serve in a Coupe glass,"[1 1⁄2 shot Rutte Dry Gin, 3⁄4 shot Lillet Bla...",SHAKE all ingredients with ice and fine strain...,11,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...
3,A.B.C. Cocktail,Lemon zest twist & Luxardo Maraschino cherry,Serve in a Nick & Nora glass,"[7 fresh Mint leaves, 1 shot Tawny port, 1 sho...",TEAR mint and place in shaker. Add other ingre...,5,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...
4,Absinthe Cocktail,Mint leaf,Serve in a Coupe glass,"[1 shot La Fée Parisienne absinthe, 2 1⁄2 shot...",SHAKE all ingredients with ice and fine strain...,5,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...
5,Absinthe Frappé,Mint sprigs bouquet,Serve in an Old-fashioned glass,"[1 shot La Fée Parisienne absinthe, 1⁄3 shot A...",SHAKE all ingredients with ice and fine strain...,1,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...
...,...,...,...,...,...,...,...,...,...
4995,Greek Daiquiri,Orange zest twist,Serve in a Coupe glass\nPhotographed in a Retr...,"[1 2⁄3 shot O/Purist Tsipouro, 2⁄3 shot Lime j...",SHAKE all ingredients with ice and fine strain...,0,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...
4996,Cazador,Lime zest twist or dehydrated lime wheel slice,Serve in a Coupe glass,"[2 shot Bacardi 8 year old rum, 2⁄3 shot Lime ...",SHAKE all ingredients with ice and fine strain...,5,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...
4997,Spiced Alexander,Dust with freshly grated nutmeg,Serve in a Coupe glass\nPhotographed in an UB ...,[1 1⁄2 shot Black Tears cacao & coffee spiced ...,SHAKE all ingredients with ice and fine strain...,1,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...
4998,Sweet Liberty's Sherry Cobbler,Fresh raspberries & mint sprig bouquet,Serve in a Wine glass,"[2 1⁄2 shot Fino sherry, 1 shot Pink grapefrui...",SHAKE all ingredients with ice and strain into...,13,https://www.diffordsguide.com/cocktails/recipe...,diffords,https://cdn.diffords.com/contrib/stock-images/...


In [4]:
import requests
links = []

def validate(start, stop):
    for i in range(start,stop):
        r = requests.get(f'https://www.diffordsguide.com/cocktails/recipe/{i}/')
        if r.status_code != 404:
            links.append(i)

import threading
threads, links = [], []
for i in range(13000,15000,20):
    t = threading.Thread(target=validate, args=(i,i+20))
    threads.append(t)
    t.start()
    t.join()
pickle.dump(links,open('./difford_13000_15000.p','wb'))

KeyboardInterrupt: 